In [12]:
import json
import os
import pandas as pd
import openai

## Create jsonl file from csv file

OpenAI fine tuning requires fine tuning data to be jsonl file

where messages are int the same format as in completion API

In [3]:
df = pd.read_csv('python_qa.csv')

In [4]:
df.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body,ParentId,Answer
0,11060,912.0,2008-08-14T13:59:21Z,NaN,18,How should I unit test a code-generator?,This is a difficult and open-ended question I ...,11060,I started writing up a summary of my experienc...
1,17250,394.0,2008-08-20T00:16:40Z,NaN,24,Create an encrypted ZIP file in Python,I'm creating an ZIP file with ZipFile in Pytho...,17250,I created a simple library to create a passwor...
2,31340,242853.0,2008-08-27T23:44:47Z,NaN,71,"How do threads work in Python, and what are co...",I've been trying to wrap my head around how th...,31340,"Yes, because of the Global Interpreter Lock (G..."
3,34020,3561.0,2008-08-29T05:43:16Z,NaN,17,Are Python threads buggy?,A reliable coder friend told me that Python's ...,34020,Python threads are good for concurrent I/O pro...
4,34570,577.0,2008-08-29T16:10:41Z,2011-11-08T16:11:43Z,13,What is the best quick-read Python book out th...,I am taking a class that requires Python. We w...,34570,"I loved Dive Into Python, especially if you're..."


In [5]:
questions, answers = df['Body'], df['Answer']

In [6]:
questions.head()

0    This is a difficult and open-ended question I ...
1    I'm creating an ZIP file with ZipFile in Pytho...
2    I've been trying to wrap my head around how th...
3    A reliable coder friend told me that Python's ...
4    I am taking a class that requires Python. We w...
Name: Body, dtype: object

In [7]:
answers.head()

0    I started writing up a summary of my experienc...
1    I created a simple library to create a passwor...
2    Yes, because of the Global Interpreter Lock (G...
3    Python threads are good for concurrent I/O pro...
4    I loved Dive Into Python, especially if you're...
Name: Answer, dtype: object

In [19]:
qa_openai_format = [{'messages': [{'role': 'user', 'content': q}, {'role':'assistant', 'content': a}]} for q, a in zip(questions, answers)]

In [21]:
print(qa_openai_format[1])

{'messages': [{'role': 'user', 'content': 'I\'m creating an ZIP file with ZipFile in Python 2.5, it works ok so far:\n\nimport zipfile, os\n\nlocfile = "test.txt"\nloczip = os.path.splitext (locfile)[0] + ".zip"\nzip = zipfile.ZipFile (loczip, "w")\nzip.write (locfile)\nzip.close()\n\n\nbut I couldn\'t find how to encrypt the files in the ZIP file.\nI could use system and call PKZIP -s, but I suppose there must be a more "Pythonic" way.  I\'m looking for an open source solution.\n'}, {'role': 'assistant', 'content': 'I created a simple library to create a password encrypted zip file in python. - here\n\nimport pyminizip\n\ncompression_level = 5 # 1-9\npyminizip.compress("src.txt", "dst.zip", "password", compression_level)\n\n\nThe library requires zlib.\n\nI have checked that the file can be extracted in WINDOWS/MAC.\n'}]}


In [22]:
len(qa_openai_format)

4429

In [31]:
with open("training_data.jsonl", "w") as f:
    for entry in qa_openai_format[:dataset_size]:
        f.write(json.dumps(entry) + "\n")

## Connect to openAI API

In [27]:
openai.api_key = os.getenv('OPENAI_API_KEY')
client = openai.Client()

## Fine tune model

In [ ]:
client.files.create(
    file=open('training_data.jsonl', 'rb'),
    purpose='fine-tune'
)

In [ ]:
client.fine_tuning.jobs.create(
    training_file='--- id of uploaded file ---',
    model='gpt-4o-mini-2024-07-18'
)

In [ ]:
client.fine_tuning.jobs.retrieve('--- job id ---')

## Compare responses

In [57]:
prompt = 'What are good python books?'

In [58]:
response = client.chat.completions.create(
            model='gpt-4o-mini',
            messages=[
                {
                    'role': 'user',
                    'content': prompt
                }
            ],
            temperature=0.7,
            max_tokens=512,
            top_p=1.0,
            frequency_penalty=0,
            presence_penalty=0
        )

In [61]:
print(response.choices[0].message.content)

There are many excellent books for learning Python, catering to different skill levels and areas of interest. Here are some highly recommended titles:

### For Beginners:
1. **"Automate the Boring Stuff with Python" by Al Sweigart**  
   - Focuses on practical programming for total beginners, teaching how to automate everyday tasks.

2. **"Python Crash Course" by Eric Matthes**  
   - A hands-on introduction to Python, covering basics and providing projects to apply your knowledge.

3. **"Head First Python" by Paul Barry**  
   - An engaging introduction to Python that uses a visual format to help learners understand concepts.

### Intermediate:
4. **"Fluent Python" by Luciano Ramalho**  
   - Offers a deeper understanding of Python's features and libraries, focusing on writing idiomatic Python code.

5. **"Effective Python: 90 Specific Ways to Write Better Python" by Brett Slatkin**  
   - Contains practical tips and best practices for writing more effective and efficient Python code.

In [ ]:
fine_tuned_response = client.chat.completions.create(
            model='--- id of a fine tuned model ---',
            messages=[
                {
                    'role': 'user',
                    'content': prompt
                }
            ],
            temperature=0.7,
            max_tokens=512,
            top_p=1.0,
            frequency_penalty=0,
            presence_penalty=0
        )

In [63]:
print(fine_tuned_response.choices[0].message.content)

There is a lot of good Python literature. It really depends on what you want to learn.

If you want to learn the basics of programming, I highly recommend:

How to Think Like a Computer Scientist: Learning with Python
by Allen B. Downey, Jeffrey Elkner, and Chris Meyers


If you want to learn the basics of Python, I recommend:

Learning Python
by Mark Lutz and David Ascher


If you want to learn more advanced Python, I suggest:

Python in a Nutshell
by Alex Martelli, Anna Ravenscroft, and Steve Holden


If you want to learn about how to think Pythonically, I propose:

Python Cookbook
by Alex Martelli, Anna Ravenscroft, and David Ascher


If you want to get into the internals of the language, I suggest:

Programming Python
by Mark Lutz


If you want to learn about specific modules, I recommend the O'Reilly series of Python Module of the Week.


If you want to learn scientific Python, I suggest:

Scientific Python
by John M. Stewart


If you want to learn about Django, I recommend:

Djan